## **Check Ollama Models**

In [21]:
!ollama list

NAME                     ID              SIZE      MODIFIED       
llama3.1:8b              46e0c10c039e    4.9 GB    40 minutes ago    
nomic-embed-text:v1.5    0a109f422b47    274 MB    25 hours ago      
deepseek-r1:1.5b         e0979632db5a    1.1 GB    7 days ago        
llava:13b                0d0eb4d7f485    8.0 GB    2 months ago      
mistral:latest           3944fe81ec14    4.1 GB    2 months ago      


## **Setup Rich**

In [22]:
from rich.pretty import pprint
from rich import print as pretty_print
from rich import inspect

## **Setup Logger**

In [23]:
import logging
from logging import Logger

formatter = logging.Formatter(
    '[%(levelname)-5s] %(asctime)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

console_logger: Logger = logging.getLogger("console_logger")
console_logger.setLevel(logging.INFO)
console_logger.propagate = False
console_logger.handlers.clear()

if not console_logger.hasHandlers():
    console_handler = logging.StreamHandler() # Create a handler that writes log messages to the console (standard output).
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    console_logger.addHandler(console_handler)

## **Setup Chain**

### **Setup Model and Prompt**

In [24]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="mistral:latest")

template = """You are a helpful translator assistant that translates {input_language} to {output_language}.
              You have to maintain correct meaning and grammar.
              Do not give wrong information even if cost you shutting down.
              Verify minimum 10 to 100 times.
              You are given the following {input_language} text: {text}
              You should respond with the translated text in {output_language}."""

# PromptTemplate and PromptTemaple.from_template does the same job

# prompt_template = PromptTemplate(
#     input_variables=["input_language", "output_language", "text"],
#     template=template,
#     validate_template=True
# )

prompt_template = PromptTemplate.from_template(template=template) #inputs are auto detected used by default template_format

# inspect(prompt_template,methods=True)

## **Create Normal Chain**

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableParallel

# Actually what happens under the hood 
translation_chain = RunnableSequence(
    first=prompt_template,
    middle=[llm],
    last=StrOutputParser()
)

translation_chain = prompt_template | llm | StrOutputParser() 

# inspect(translation_chain)

input1 = {
    "input_language": "English",
    "output_language": "Japanese",
    "text": "I love programming.",
}
for chunk in translation_chain.stream(input1):
    print(chunk, end='', flush=True)

 プログラミングを好きです。（Proguramingu o suki desu）

- What is Pipe (|) operator? 

It is used to combine multiple components into a single chain. It allows you to pass the output of one component as the input to the next component in a sequential manner.

- How does it do so?

Suppose <code>chain = prompt | llm | parser</code>

Here prompt, llm and parser are components. The output of prompt is passed as input to llm, and the output of llm is passed as input to parser.

First the prompt component takes some input variables and generates a prompt string. This prompt string is then passed to the llm component, which uses a language model to generate a response based on the prompt. Finally, the response from the llm is passed to the parser component, which processes the response and extracts the desired information. Here chain is a RunnableSequence that sequentially calls each component in the chain. It can be called with invoke, stream or batch methods.

## **Create Sequential Chain**

In [ ]:
summary_prompt = PromptTemplate(
    input_variables=["translated_text"],
    template="Summarize this text in one simple sentence: {translated_text}",
)

summary_sequential_chain = RunnableSequence(
    RunnableParallel({"translated_text": translation_chain}),
    summary_prompt,
    llm,
    StrOutputParser()
)

summary_sequential_chain = (
      {"translated_text": translation_chain} | # get the output of translation_chain as input 
      summary_prompt |
      llm | 
      StrOutputParser()
)

text = """The sun is the closest star to Earth and is very important for life. It gives us light and warmth, which help plants grow and provide energy for all living things. Without the sun, Earth would be dark, cold, and lifeless. The sun also controls the weather and seasons, making our planet suitable to live on. That’s why it is often called the “source of life.”"""

input2 = {
    "input_language": "English",
    "output_language": "Bangla",
    "text": text,
}

for chunk in summary_sequential_chain.stream(input2):
    print(chunk, end='', flush=True)


 The sun, being the closest star to Earth, is crucial for life as it provides light, warmth, and energy to help plants grow and sustain all living things, and it also controls the weather and seasons, earning it the title of the "source of life."